Script to read ocean bathymetry data and plot horizontal gradients.

In [7]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from netCDF4 import Dataset
import pandas as pd
import xarray as xr
import metpy.calc as mpcalc
from metpy.units import units
%matplotlib qt

In [9]:
file_path = '/archive/Raphael.Dussin/xanadu_esm4_20190304_mom6_2019.08.08/OM4p125_JRA55do1.4_cycle1/gfdl.ncrc4-intel16-prod/pp/ocean_annual_z_d2/';
file_name = 'ocean_annual_z_d2.static.nc';

In [11]:
#fil = Dataset(file_path+file_name, 'r')
fil = xr.open_dataset(file_path+file_name)
data =  fil.metpy.parse_cf()

Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_lon

In [67]:
print(data['deptho'])

<xarray.DataArray 'deptho' (yh: 2240, xh: 2880)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Coordinates:
    crs      object Projection: latitude_longitude
  * xh       (xh) float64 -298.6 -298.5 -298.4 -298.3 ... 61.05 61.16 61.28
  * yh       (yh) float64 -83.62 -83.58 -83.54 -83.5 ... 89.81 89.86 89.92 89.97
Attributes:
    long_name:      Sea Floor Depth
    units:          m
    cell_methods:   area:mean yh:mean xh:mean time: point
    cell_measures:  area: areacello
    standard_name:  sea_floor_depth_below_geoid


In [70]:
# Using netcdf directly
#f.variables
#lat = f.variables['yh'];
#lon = f.variables['xh'];
#depth = f.variables['deptho'];

# using xarray
#lat = f.yh;
#lon = f.xh;
#depth = f.deptho;

# using metpy
x, y = data['deptho'].metpy.coordinates('x', 'y');
depth = data['deptho'];
lat, lon = xr.broadcast(y, x);
f = mpcalc.coriolis_parameter(lat)

In [23]:
depth_crs = data['deptho'].metpy.cartopy_crs;
dx, dy = mpcalc.lat_lon_grid_deltas(lon, lat, initstring=depth_crs.proj4_init)
#[depth_y, depth_x] = np.gradient(depth[:,:], lat[:], lon[:])

In [60]:
depth_y, depth_x = mpcalc.gradient(depth, deltas=(dy, dx))

In [93]:
#print(depth)
#print(lat.shape);
#print(lon.shape);
#print(depth_x.shape);
print(lat[2239,:]);
print(dx[2239,:])
#print(dx1[2200:,2000])
#plt.plot(x.diff(1))
#plt.plot(y.diff(1))

<xarray.DataArray 'yh' (xh: 2880)>
array([89.97297808, 89.97297808, 89.97297808, ..., 89.97297808,
       89.97297808, 89.97297808])
Coordinates:
    crs      object Projection: latitude_longitude
    yh       float64 89.97
  * xh       (xh) float64 -298.6 -298.5 -298.4 -298.3 ... 61.05 61.16 61.28
Attributes:
    long_name:       h point nominal latitude
    units:           degrees_north
    cartesian_axis:  Y
    _metpy_axis:     Y
[6.12971213 6.12997384 6.13023443 ... 6.12865448 6.1289205  6.12918546] meter


In [131]:
fig, axs = plt.subplots(nrows = 4, ncols = 1, sharex = True)
pcm = axs[0].contourf(x,y,depth[:,:], np.arange(0,6500,500), extend = 'both', cmap = cm.terrain)
fig.colorbar(pcm, ax = axs[0])
axs[0].set_ylabel('lat')
axs[0].set_ylim(-80, -30)
axs[0].set_title('Depth (m)')

pcm = axs[1].contourf(x,y,-f*depth_x[:,:], np.arange(-10**-5, 1.1*10**-5, 10**-6), extend = 'both', cmap = cm.Spectral)
fig.colorbar(pcm, ax=axs[1])
axs[1].set_ylabel('lat')
axs[1].set_title('f*d/dx(Topography)')
axs[1].set_ylim(-80, -30)

pcm = axs[2].contourf(x,y,-f*depth_y[:,:], np.arange(-10**-5, 1.1*10**-5, 10**-6), extend = 'both', cmap = cm.Spectral)
fig.colorbar(pcm, ax = axs[2])
axs[2].set_ylabel('lat')
axs[2].set_title('f*d/dy(Topography)')
axs[2].set_ylim(-80, -30)

pcm = axs[3].contourf(x,y,np.sqrt(f**2*depth_x**2 + f**2*depth_y**2), np.arange(0, 2.1*10**-5, 2*10**-6), cmap = cm.BuGn)
fig.colorbar(pcm,ax=axs[3])
axs[3].set_ylabel('lat')
axs[3].set_xlabel('lon')
axs[3].set_title('Absolute gradient term')
axs[3].set_ylim(-80, -30)

(-80, -30)

In [133]:
fig.savefig('/home/Hemant.Khatri/Work/Topography_southern_ocean1.eps', format = 'eps', dpi=300)

In [134]:
2*np.pi*6400*1000*np.cos(70*np.pi/180)/2880

4775.502087340665

In [126]:
np.nanmean(depth_x)

-7.903880608716085e-06

In [139]:
data.close()